In [ ]:
IMAGE_SIZE=320
BATCH_SIZE=16
EPOCHS=10

In [ ]:
!pip install ultralytics

In [ ]:
import os
import requests
import yaml
import zipfile
import shutil
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
yaml_url = 'https://github.com/JeremyTubongbanua/blind_person_assistant/raw/jt/headset/custom_data.yaml'
custom_data_file_name = '/content/custom_data.yaml'

response = requests.get(yaml_url)
response.raise_for_status()  # Raise an exception for bad status codes

with open(custom_data_file_name, 'w') as f:
  f.write(response.text)

print(f"YAML file downloaded and saved to {custom_data_file_name}")

YAML file downloaded and saved to /content/custom_data.yaml


In [ ]:
zip_url = "https://github.com/JeremyTubongbanua/blind_person_assistant/raw/jt/headset/dataset.zip"
dataset_file_name = "/content/dataset.zip"

def download_zip(url, filename):
    """Downloads a file from a URL and saves it to the specified filename if it doesn't already exist."""
    if os.path.exists(filename):
      os.remove(filename)
      print(f"Removed {filename}")

    response = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded {filename}")

download_zip(zip_url, dataset_file_name)

Downloaded /content/dataset.zip


In [ ]:
folder_name = "./"

with zipfile.ZipFile(dataset_file_name, "r") as zip_ref:
  zip_ref.extractall(folder_name)
  print(f"Dataset extracted to {folder_name} folder.")

In [ ]:
def remove_mac_folders(root_dir):
  for dirpath, dirnames, filenames in os.walk(root_dir):
    for dirname in dirnames:
      if dirname == '__MACOSX':
        full_path = os.path.join(dirpath, dirname)
        try:
          shutil.rmtree(full_path)
          print(f"Removed: {full_path}")
        except Exception as e:
          print(f"Error removing {full_path}: {e}")

remove_mac_folders('/content')

In [ ]:
!yolo train model=yolov8n.pt data={custom_data_file_name} epochs=EPOCHS imgsz=IMAGE_SIZE batch=BATCH_SIZE

In [ ]:
!yolo val model=/content/runs/detect/train/weights/best.pt data={custom_data_file_name} imgsz=IMAGE_SIZE

In [ ]:
!yolo predict model=/content/runs/detect/train/weights/best.pt source=/content/dataset/images/test/reduced_IMG_4651.JPG

In [ ]:
!cp /content/runs/detect/train/weights/best.pt /content/yolov8ntrained.pt

In [ ]:
!yolo export model=/content/yolov8ntrained.pt format=onnx

In [ ]:
!pip install blobconverter

In [ ]:
SHAVES=10

import blobconverter
blob_path = blobconverter.from_onnx(
    model="/content/yolov8ntrained.onnx",
    data_type="FP16",
    shaves=SHAVES,
)
blob_path

In [ ]:
import shutil

source_path = f'/root/.cache/blobconverter/yolov8ntrained_openvino_2022.1_{SHAVES}shave.blob'
destination_path = f'/content/yolov8n_openvino_2022.1_{SHAVES}shave.blob'

try:
  shutil.move(source_path, destination_path)
  print(f"File moved successfully from {source_path} to {destination_path}")
except FileNotFoundError:
  print(f"File not found at {source_path}")
except Exception as e:
  print(f"An error occurred: {e}")

In [ ]:
from google.colab import files
files.download('/content/yolov8ntrained.pt')
files.download('/content/yolov8ntrained.onnx')
files.download(f'/content/yolov8n_openvino_2022.1_{SHAVES}shave.blob')